In [ ]:
import gpt4all
import pandas as pd
import re
from gpt4all import GPT4All

In [23]:
def load_and_process_data(file_path):
    
    df = pd.read_csv(file_path, delimiter="\t", header=None)
    
    df.columns = ["Question", "Option_A", "Option_B", "Option_C", "Option_D", "Correct_Answer"]

    return df


def ask_zero_shot(model, question, options):
    prompt = "Answer the following question with a single letter:\n"
    prompt += f"Question: {question}\n"
    prompt += f"A: {options[0]}\n"
    prompt += f"B: {options[1]}\n"
    prompt += f"C: {options[2]}\n"
    prompt += f"D: {options[3]}\n"
    prompt += "Answer: "
    print(prompt)
    response = model.generate(prompt)
    print(response)
    match = re.search(r"\b([ABCD])\b", response)
    if match:
        return response, match.group(1)
    else:
        return response, None

def ask_few_shot(model, question, options):
    prompt = f"""The following are two examples of multiple choice questions (with answers) about astronomy:
    
Question: Which of the following characteristics would not necessarily suggest that a rock we found is a meteorite?
A: It has a fusion crust
B: It contains solidified spherical droplets
C: It is highly processed
C: It has different elemental composition than earth
Answer: C

Question: Which is not a similarity between Saturn and Jupiter's atmospheres?
A: a composition dominated by hydrogen and helium
B: the presence of belts zones and storms
C: an equatorial wind speed of more than 900 miles per hour
D: significant "shear" between bands of circulation at different latitudes
Answer: C

Now, answer the following question with a single letter:
\n"""
    
    prompt += f"Question: {question}\n"
    prompt += f"A: {options[0]}\n"
    prompt += f"B: {options[1]}\n"
    prompt += f"C: {options[2]}\n"
    prompt += f"D: {options[3]}\n"
    prompt += "Answer: "
    print(prompt)
    response = model.generate(prompt)
    print(response)
    match = re.search(r"\b([ABCD])\b", response)
    if match:
        return response, match.group(1)
    else:
        return response, None

def run_prompt_strategy(df, model, prompt_strategy):
    results = []

    for _, row in df.iterrows():
        question = row["Question"]
        options = [row["Option_A"], row["Option_B"], row["Option_C"], row["Option_D"]]
        correct_answer = row["Correct_Answer"]
        if prompt_strategy == "zero-shot":
            response, answer = ask_zero_shot(model, question, options)
        elif prompt_strategy == "few-shot":
            response, answer = ask_few_shot(model, question, options)
        else:
            raise ValueError("Invalid prompt strategy")
        
        results.append({
            "Question": question,
            "Option A": options[0],
            "Option B": options[1],
            "Option C": options[2],
            "Option D": options[3],
            "Correct Answer": correct_answer,
            "Predicted Answer": answer,
            "Full Response": response
        })
    return pd.DataFrame(results)
    

In [24]:
testset_path = "TestSet.txt"
df_testset = load_and_process_data(testset_path)
df_additional_data = load_and_process_data("AdditionalDataForFewShotPrompting.txt")
# Test prompt strategy on one example
model_name = "Meta-Llama-3-8B-Instruct.Q4_0.gguf"
model = GPT4All(model_name)

question = df_testset["Question"][0]
options = df_testset.iloc[0, 1:5].values
correct_answer = df_testset["Correct_Answer"][0]
response, predicted_answer = ask_few_shot(model, question, options)
print(f"Correct Answer: {correct_answer}")
print(f"Predicted Answer: {predicted_answer if predicted_answer else response}")

The following are two examples of multiple choice questions (with answers) about astronomy:
    
Question: Which of the following characteristics would not necessarily suggest that a rock we found is a meteorite?
A: It has a fusion crust
B: It contains solidified spherical droplets
C: It is highly processed
C: It has different elemental composition than earth
Answer: C

Question: Which is not a similarity between Saturn and Jupiter's atmospheres?
A: a composition dominated by hydrogen and helium
B: the presence of belts zones and storms
C: an equatorial wind speed of more than 900 miles per hour
D: significant "shear" between bands of circulation at different latitudes
Answer: C

Now, answer the following question with a single letter:

Question: What is true for a type-Ia ("type one-a") supernova?
A: This type occurs in binary systems.
B: This type occurs in young galaxies.
C: This type produces gamma-ray bursts.
D: This type produces high amounts of X-rays.
Answer: 
 A

Please note t

In [26]:
model_name = "Meta-Llama-3-8B-Instruct.Q4_0.gguf"
model = GPT4All(model_name)
Llama_zero_shot_results = run_prompt_strategy(df_testset, model, "zero-shot")
Llama_zero_shot_results.to_csv("Llama_zero_shot_results.csv", index=False)
Llama_few_shot_results = run_prompt_strategy(df_testset, model, "few-shot")
Llama_few_shot_results.to_csv("Llama_few_shot_results.csv", index=False)

model_name = "Nous-Hermes-2-Mistral-7B-DPO.Q4_0.gguf"
model = GPT4All(model_name)
Hermes_zero_shot_results = run_prompt_strategy(df_testset, model, "zero-shot")
Hermes_zero_shot_results.to_csv("Hermes_zero_shot_results.csv", index=False)
Hermes_few_shot_results = run_prompt_strategy(df_testset, model, "few-shot")
Hermes_few_shot_results.to_csv("Hermes_few_shot_results.csv", index=False)

Answer the following question with a single letter:
Question: What is true for a type-Ia ("type one-a") supernova?
A: This type occurs in binary systems.
B: This type occurs in young galaxies.
C: This type produces gamma-ray bursts.
D: This type produces high amounts of X-rays.
Answer: 
 A

Explanation: Type-Ia (or "type one-a") supernovae are a class of stellar explosions that occur when a white dwarf star accumulates material from a companion star and reaches a critical mass, causing it to explode. They typically originate in binary systems where the white dwarf is part of an interacting pair with another star. This type of supernova is often used as a "standard candle" for measuring distances in astronomy because they have consistent maximum brightnesses. The other options are not correct: Type-Ia supernovae do not occur in young galaxies, and while some gamma-ray bursts may be associated with supernovae, this is not specific to type Ia events. Finally, X-rays are produced by many t

Downloading: 100%|██████████| 4.11G/4.11G [02:17<00:00, 29.9MiB/s]
Verifying: 100%|██████████| 4.11G/4.11G [00:06<00:00, 672MiB/s]


Answer the following question with a single letter:
Question: What is true for a type-Ia ("type one-a") supernova?
A: This type occurs in binary systems.
B: This type occurs in young galaxies.
C: This type produces gamma-ray bursts.
D: This type produces high amounts of X-rays.
Answer: 
 A
Explanation: Type Ia supernovae occur when a white dwarf star accretes matter from its companion until it reaches the Chandrasekhar limit and undergoes an explosion, typically in binary systems where one component is a white dwarf and another is a main-sequence or giant star. They do not produce gamma-ray bursts nor high amounts of X-rays; they are most often observed in galaxies like our own Milky Way (not young galaxies).
Answer the following question with a single letter:
Question: If you know both the actual brightness of an object and its apparent brightness from your location then with no other information you can estimate:
A: Its speed relative to you
B: Its composition
C: Its size
D: Its dist